<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/eat_drink_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 飲食店
- レストラン
- レストラン(チェーン店) 
- カフェ
- カフェ(チェーン店) 
- 飲み屋
- ファストフード
- ネットカフェ

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [3]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [4]:
df_navitime_org = pd.read_csv(os.path.join(DIR_NAME, 'navitime_master.csv'))
print(df_navitime_org.shape)

df_netcafe = df_navitime_org[df_navitime_org['データ元'] == 'ネットカフェ'].reset_index(drop=True)
df_netcafe['施設分類'] = 'ネットカフェ'
print(df_netcafe.shape)

(7854, 7)
(185, 8)


In [5]:
df_restaurant_org = pd.read_csv(os.path.join(DIR_NAME, 'restaurant_nearby.csv'))
print(df_restaurant_org.shape)

df_cafe_org = pd.read_csv(os.path.join(DIR_NAME, 'cafe_nearby.csv'))
print(df_cafe_org.shape)

df_nightclub_bar_org = pd.read_csv(os.path.join(DIR_NAME, 'nightclub_bar_nearby.csv'))
print(df_nightclub_bar_org.shape)

(10366, 7)
(6575, 7)
(8690, 7)


In [6]:
df_eat_drink = pd.concat([
    df_restaurant_org,
    df_cafe_org,
    df_nightclub_bar_org
]).drop_duplicates(subset=['place_id']).reset_index(drop=True)
print(df_eat_drink.shape)

(24674, 7)


In [7]:
df_eat_drink['types_list'] = df_eat_drink['types'].apply(lambda x: x.replace('"', '').replace('[', '').replace(']', '').replace("'", '').replace(' ', '').split(','))
types_list = df_eat_drink['types_list'].explode()

In [8]:
store_types = ['レストラン', 'レストラン(チェーン店)', 'カフェ', 'カフェ(チェーン店)', '飲み屋', 'カラオケ', 'ファストフード', 'ネットカフェ']
for t in store_types:
    df_eat_drink[t] = 0

In [9]:
fast_food_stores = [
    'マクドナルド', 'ケンタッキー', 'サブウェイ', 'モスバーガー', 'ロッテリア', 'ファーストキッチン', 'フレッシュネスバーガー', 
    'バーガーキング', 'ドムドム', 'ミスタードーナツ', 'ピザハット', 'ドミノ・ピザ', 'ピザーラ', 'ほっかほっか亭', 'ほっともっと',
    '壱番屋', '吉野家', 'すき家', 'なか卯', '松屋', 'ほかほか弁当', '築地銀だこ'
]
for store in fast_food_stores:
    df_eat_drink.loc[df_eat_drink['name'].str.contains(store), 'ファストフード'] = 1

In [10]:
cafe_chain_store = [
    'スターバックス', "TULLY'S", 'タリーズ', 'コメダ珈琲店', 'ドトール', 'カフェ・ド・クリエ', '上島珈琲店', 
    'イタリアン・トマト', 'サンマルクカフェ', 'エクセルシオール', 'ルノアール', '星乃珈琲店',
    'PRONTO', 'コージーコーナー', 'カフェ コロラド', '珈琲館', 'カフェ・ベローチェ', 'カフェ・ド・クリエ'
]
for store in cafe_chain_store:
    df_eat_drink.loc[df_eat_drink['name'].str.contains(store), 'カフェ(チェーン店)'] = 1

In [11]:
restaurant_chain_store = [
    'サイゼリヤ', 'ガスト', '丸亀製麺', '王将', 'ジョナサン', 'デニーズ', '養老乃瀧', 'バーミヤン', '夢庵',
    '不二家レストラン', 'はま寿司', 'スシロー', 'くら寿司', 'ビッグボーイ', '木曽路', '一風堂', 'ロイヤルホスト',
    'ジョリーパスタ', '大戸屋', '天下一品', '鳥貴族', '白木屋', 'ココス', 'びっくりドンキー', 'やよい軒',
    'ジョナサン', '和食さと', '華屋与兵衛', 'ジョイフル', '日高屋'
]
for store in restaurant_chain_store:
    df_eat_drink.loc[df_eat_drink['name'].str.contains(store), 'レストラン(チェーン店)'] = 1

In [12]:
netcafe_chain_store = [
    '快活CLUB', '自遊空間', 'コミック・バスター', '宝島24'
]
for store in netcafe_chain_store:
    df_eat_drink.loc[df_eat_drink['name'].str.contains(store), 'ネットカフェ'] = 1

In [13]:
df_eat_drink.loc[df_eat_drink['types_list'].apply(lambda x: 'restaurant' in x), 'レストラン'] = 1
df_eat_drink.loc[df_eat_drink['types_list'].apply(lambda x: 'cafe' in x), 'カフェ'] = 1
df_eat_drink.loc[df_eat_drink['types_list'].apply(lambda x: 'bar' in x), '飲み屋'] = 1
df_eat_drink.loc[df_eat_drink['types_list'].apply(lambda x: 'night_club' in x), '飲み屋'] = 1
df_eat_drink.loc[df_eat_drink['name'].str.contains('カラオケ'), 'カラオケ'] = 1
df_eat_drink['sum'] = df_eat_drink[store_types].sum(axis=1)

In [14]:
df_eat_drink['施設分類'] = None

df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['ネットカフェ'] == 1), '施設分類'] = 'ネットカフェ'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['ファストフード'] == 1), '施設分類'] = 'ファストフード'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['カフェ(チェーン店)'] == 1), '施設分類'] = 'カフェ(チェーン店)'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['レストラン(チェーン店)'] == 1), '施設分類'] = 'レストラン(チェーン店)'

df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['ファストフード'] == 1)&(df_eat_drink['飲み屋'] == 0), '施設分類'] = 'ファストフード'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['レストラン'] == 1)&(df_eat_drink['飲み屋'] == 0), '施設分類'] = 'レストラン'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['カフェ'] == 1)&(df_eat_drink['飲み屋'] == 0), '施設分類'] = 'カフェ'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['カラオケ'] == 1), '施設分類'] = 'カラオケ'
df_eat_drink.loc[(df_eat_drink['施設分類'].isna())&(df_eat_drink['飲み屋'] == 1), '施設分類'] = '飲み屋'

assert len(df_eat_drink[df_eat_drink['施設分類'].isna()]) == 0

In [15]:
df_eat_drink['施設分類'].value_counts()

レストラン           9769
飲み屋             8328
カフェ             4527
カフェ(チェーン店)       695
ファストフード          662
カラオケ             396
レストラン(チェーン店)     279
ネットカフェ            18
Name: 施設分類, dtype: int64

In [16]:
df_eat_drink_master = df_eat_drink[
    (df_eat_drink['施設分類'].notna()) &
    (df_eat_drink['施設分類'] != 'ネットカフェ') &
    (df_eat_drink['施設分類'] != 'カラオケ')
][['name', 'vicinity', 'lat', 'lng', '施設分類', 'rating']].rename(columns={
    'name': '名称',
    'vicinity': '住所',
    'lat': '緯度', 
    'lng': '経度'
})
print(df_eat_drink_master.shape)

(24260, 6)


In [17]:
df_eat_drink_master = pd.concat([
    df_eat_drink_master,
    df_netcafe[['名称', '住所', '緯度', '経度', '施設分類']]
]).sort_values(['施設分類', '住所']).reset_index(drop=True)
print(df_eat_drink_master.shape)

(24445, 6)


In [18]:
df_eat_drink_master.to_csv(os.path.join(DIR_NAME, 'eat_drink_master.csv'), encoding='utf-8_sig', index=False)